In [1]:
import pandas as pd
import os

df = pd.read_csv("../docs/event_objective_summary.csv")
output_folder = '../docs/'


In [2]:
df = df.drop(['Person 1_Location Of Person','Person 1.2_Reporter Organization', 'Person 1.3_Function', 'Person 1.4_Qualification', 'Person 1.5_Experience', 'Person 1.6_Cabin Activity','Person 1.10_ASRS Report Number.Accession Number','Person 2_Location Of Person', 'Person 2.1_Location In Aircraft',
       'Person 2.2_Reporter Organization', 'Person 2.3_Function',
       'Person 2.4_Qualification', 'Person 2.5_Experience',
       'Person 2.6_Cabin Activity', 'Person 2.7_Human Factors',
       'Person 2.8_Communication Breakdown',
       'Person 2.9_UAS Communication Breakdown',
       'Person 2.10_ASRS Report Number.Accession Number',
       'Events.1_Miss Distance','Report 1.1_Callback',  'Report 2.1_Callback', 'Events_Anomaly',
       'Events.2_Were Passengers Involved In Event', 'Events.3_Detector',
       'Events.4_When Detected', 'Events.5_Result',
       'Assessments_Contributing Factors / Situations',
       'Assessments.1_Primary Problem', 'Report 1_Narrative',
       'Report 2_Narrative', 'Report 1.2_Synopsis','Person 1.1_Location In Aircraft', 'Person 1.7_Human Factors'], axis=1)

In [3]:
print(df.columns)

Index(['ACN', 'Person 1.8_Communication Breakdown', 'objective_summary',
       'assigned_labels'],
      dtype='object')


In [4]:
def split_parties(row):
    try:
        parts = row.split(';')
        party1 = parts[0].replace("Party1", "").strip()
        party2 = parts[1].replace("Party2", "").strip()
        return pd.Series([party1, party2])
    except:
        return pd.Series([None, None])


df[['Party1_Role', 'Party2_Role']] = df['Person 1.8_Communication Breakdown'].apply(split_parties)


df.drop(columns=['Person 1.8_Communication Breakdown'], inplace=True)


cols = list(df.columns)
acn_index = cols.index('ACN')


new_order = (
    cols[:acn_index + 1] +
    ['Party1_Role', 'Party2_Role'] +
    [col for col in cols[acn_index + 1:] if col not in ['Party1_Role', 'Party2_Role']]
)

df = df[new_order]


df.head()

,ACN,Party1_Role,Party2_Role,objective_summary,assigned_labels
0,2068792,Flight Attendant,Flight Attendant,A flight attendant discovered an open biohazar...,"safety related, crew-related issues, operation..."
1,2100755,Flight Attendant,Ground Personnel,The incident arose from a security breach invo...,"lithium batteries, training, communication iss..."
2,2102491,Flight Attendant,Flight Attendant,The incident involved an off-duty Flight Atten...,"safety related, crew-related issues, operation..."
3,2119017,Flight Attendant,Ground Personnel,The incident involved a flight attendant who w...,"between attendant and passengers, crew-related..."
4,2168329,Flight Attendant,Ground Personnel,The incident involved a flight attendant (Part...,"lithium batteries, between attendant and crew,..."


In [5]:
output_path = os.path.join(output_folder, 'role_split_clean.csv')
df.to_csv(output_path, index=False)